In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re

In [5]:
executable_path = {'executable_path':'C:/Users/icani/Desktop/Class/work/Mars_Scraping/chromedriver'}

browser = Browser('chrome', **executable_path, headless=False)

In [6]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [7]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [8]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Establishes Board to Initially Review Mars Sample Return Plans</div>

In [9]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'NASA Establishes Board to Initially Review Mars Sample Return Plans'

In [10]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.'

### Featured Images

In [12]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [13]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [14]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')


In [15]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [16]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## Mars Facts

In [31]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [32]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [33]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
#'https://marshemispheres.com/'

browser.visit(url)

In [34]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# Parse the data
html = browser.html
urls_soup = soup(html, 'html.parser')

# 3. Write code to retrieve the image urls and titles for each hemisphere.

divs = urls_soup.find("div", class_='collapsible results')
anchors = divs.find_all('a')
relative_urls = set([anchor['href'] for anchor in anchors])
print(f'Found {len(relative_urls)} URLs')

base_url = 'https://astrogeology.usgs.gov'

for relative_url in relative_urls:
    hemispheres = {}
    
    full_url = f'{base_url}{relative_url}'
    browser.visit(full_url)
    browser.links.find_by_text('Open').click()
    
    html = browser.html
    urls_soup = soup(html, 'html.parser')
    
    downloads_div = urls_soup.find('div', class_='downloads')
    img_anchor = downloads_div.find('a', text=re.compile('Sample'))
    img_url = img_anchor['href']
    print(f'--> url: {img_url}')
    
    title_elem = urls_soup.select_one('div.content')
    title = title_elem.find("h2", class_='title').get_text()
    print(f'--> title: {title}')
    hemispheres = {
        'img_url': img_url,
        'title': title,
    }
    hemisphere_image_urls.append(hemispheres)


Found 4 URLs
--> url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
--> title: Syrtis Major Hemisphere Enhanced
--> url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
--> title: Cerberus Hemisphere Enhanced
--> url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
--> title: Valles Marineris Hemisphere Enhanced
--> url: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
--> title: Schiaparelli Hemisphere Enhanced


In [35]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'}]

In [36]:
# 5. Quit the browser
browser.quit()